# SPML Project

##### Using graph attention to learn semantic in point cloud 3D manmade structure and keep only useful/significant points

In [8]:
import numpy as np
from scipy.spatial import KDTree
import networkx as nx
import open3d as o3d
import os
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 0. Load datasets + Preprocess

In [9]:
def load_semantic3d_txt(path):
    data = np.loadtxt(path)
    points = data[:, 0:3]      # x, y, z
    features = data[:, 3:7]    # intensity, r, g, b
    return points, features

In [10]:
def load_labels(path):
    """Charge le fichier de labels (un label par ligne)"""
    labels = np.loadtxt(path, dtype=int)
    return labels

def filter_by_label(points, features, labels, target_label=5):
    """Filtre les points pour ne garder que ceux avec le label cible"""
    mask = (labels == target_label)
    filtered_points = points[mask]
    filtered_features = features[mask]
    filtered_labels = labels[mask]
    
    print(f"  Filtered: {len(points)} -> {len(filtered_points)} points (label={target_label})")
    return filtered_points, filtered_features, filtered_labels

## 1. Convert points cloud to graph

Radius estimation

In [11]:
def estimate_radius(points, k=1, scale=2.0, sample_size=10000):
    """Estime le rayon en utilisant seulement un échantillon"""
    if len(points) > sample_size:
        indices = np.random.choice(len(points), sample_size, replace=False)
        sample = points[indices]
    else:
        sample = points
    
    tree = KDTree(sample)
    dists, _ = tree.query(sample, k=k+1)
    return np.mean(dists[:, k]) * scale

Graph construction

In [12]:
def build_radius_graph_batched(points, radius, max_neighbors=None, batch_size=5000):
    """Construit le graphe par lots pour éviter les problèmes de mémoire"""
    kdtree = KDTree(points)
    graph = nx.Graph()
    
    # Ajouter tous les nœuds
    for i in range(len(points)):
        graph.add_node(i, pos=points[i].tolist())
    
    print(f"  Building graph with {len(points)} nodes...")
    
    # Traiter par lots
    num_batches = (len(points) + batch_size - 1) // batch_size
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(points))
        
        if batch_idx % 10 == 0:
            print(f"  Batch {batch_idx + 1}/{num_batches}")
        
        # Pour chaque point du lot, trouver ses voisins
        for i in range(start_idx, end_idx):
            # Requête pour ce point uniquement
            neighbors = kdtree.query_ball_point(points[i], radius)
            
            # Limiter le nombre de voisins si nécessaire
            if max_neighbors is not None and len(neighbors) > max_neighbors + 1:
                # Calculer distances et garder les k plus proches
                dists = np.linalg.norm(points[neighbors] - points[i], axis=1)
                closest = np.argpartition(dists, max_neighbors + 1)[:max_neighbors + 1]
                neighbors = [neighbors[idx] for idx in closest]
            
            # Ajouter les arêtes
            for j in neighbors:
                if i < j:  # Éviter les doublons (i,j) et (j,i)
                    dist = np.linalg.norm(points[i] - points[j])
                    graph.add_edge(i, j, weight=dist)
    
    return graph

Save graph in local

In [13]:
def save_graph(graph, path):
    with open(path, 'wb') as f:
        pickle.dump(graph, f, protocol=pickle.HIGHEST_PROTOCOL)

Load it

In [14]:
def load_graph(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

Load files and convert it into graph

In [15]:
def process_files(
    file_list,
    input_dir,
    labels_dir,
    output_dir,
    target_label=5,
    max_neighbors=10,
    radius_scale=2.0,
    batch_size=5000,
):
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in file_list:
        input_path = os.path.join(input_dir, filename)
        
        label_filename = filename.replace(".txt", ".labels")
        label_path = os.path.join(labels_dir, label_filename)
        
        output_path = os.path.join(
            output_dir,
            filename.replace(".txt", ".graph")
        )
        
        if os.path.exists(output_path):
            print(f"[SKIP] {filename} (graph already exists)")
            continue
        
        print(f"[PROCESS] {filename}")
        
        # 1. Load data
        points, features = load_semantic3d_txt(input_path)
        print(f"  Loaded {len(points)} points")
        
        # 2. Load labels
        if not os.path.exists(label_path):
            print(f"  WARNING: Label file not found: {label_path}")
            print(f"  Skipping {filename}")
            continue
        
        labels = load_labels(label_path)
        
        if len(labels) != len(points):
            print(f"  ERROR: Mismatch between points ({len(points)}) and labels ({len(labels)})")
            continue
        
        # 3. Filter by label
        filtered_points, filtered_features, filtered_labels = filter_by_label(
            points, features, labels, target_label=target_label
        )
        
        if len(filtered_points) == 0:
            print(f"  WARNING: No points with label {target_label} found. Skipping.")
            continue
        
        # 4. Estimate radius (sur échantillon)
        radius = estimate_radius(filtered_points, k=1, scale=radius_scale)
        print(f"  Estimated radius: {radius:.4f}")
        
        # 5. Build graph with batches
        graph = build_radius_graph_batched(
            filtered_points,
            radius=radius,
            max_neighbors=max_neighbors,
            batch_size=batch_size
        )
        print(f"  Graph built: {graph.number_of_nodes()} nodes, {graph.number_of_edges()} edges")
        
        # 6. Attach node features
        for i in range(len(filtered_points)):
            graph.nodes[i]["feat"] = np.concatenate(
                [filtered_points[i], filtered_features[i]]
            ).tolist()
            # Optionnel: ajouter aussi le label au nœud
            graph.nodes[i]["label"] = int(filtered_labels[i])
        
        # 7. Save graph
        save_graph(graph, output_path)
        print(f"  Saved graph to {output_path}\n")

In [18]:
file_list = [
    #"MarketplaceFeldkirch_Station4_rgb_intensity-reduced.txt",
    "StGallenCathedral_station6_rgb_intensity-reduced.txt",
]

process_files(
    file_list=file_list,
    input_dir="dataset/raw_txt",
    labels_dir="dataset/labels",
    output_dir="dataset/graphs",
    target_label=5,  # Le label à garder
    max_neighbors=10,
    radius_scale=2.0,
    batch_size=5000,
)

[PROCESS] StGallenCathedral_station6_rgb_intensity-reduced.txt
  Loaded 14608690 points
  Filtered: 14608690 -> 2156724 points (label=5)
  Estimated radius: 0.6902
  Building graph with 2156724 nodes...
  Batch 1/432
  Batch 11/432
  Batch 21/432
  Batch 31/432
  Batch 41/432
  Batch 51/432
  Batch 61/432
  Batch 71/432
  Batch 81/432
  Batch 91/432
  Batch 101/432
  Batch 111/432
  Batch 121/432
  Batch 131/432
  Batch 141/432
  Batch 151/432
  Batch 161/432
  Batch 171/432
  Batch 181/432
  Batch 191/432
  Batch 201/432
  Batch 211/432
  Batch 221/432
  Batch 231/432
  Batch 241/432
  Batch 251/432
  Batch 261/432
  Batch 271/432
  Batch 281/432
  Batch 291/432
  Batch 301/432
  Batch 311/432
  Batch 321/432
  Batch 331/432
  Batch 341/432
  Batch 351/432
  Batch 361/432
  Batch 371/432
  Batch 381/432
  Batch 391/432
  Batch 401/432
  Batch 411/432
  Batch 421/432
  Batch 431/432
  Graph built: 2156724 nodes, 10774108 edges
  Saved graph to dataset/graphs\StGallenCathedral_station6_

# MODEL

In [ ]:
# Import the base class with all helper methods
from GAT.models.definitions.GAT import GATLayerImp3

class GATWithSpatial(GATLayerImp3):
    """Extend GATLayerImp3 with spatial encoding"""
    
    def __init__(self, num_in_features, num_out_features, num_of_heads, 
                 concat=True, activation=nn.ELU(), dropout_prob=0.6, 
                 add_skip_connection=True, bias=True):
        
        # Initialize parent class - gets you ALL the helper methods
        super().__init__(
            num_in_features, num_out_features, num_of_heads,
            concat, activation, dropout_prob, add_skip_connection, bias
        )
        
        # ADD YOUR SPATIAL COMPONENTS
        self.spatial_weight = nn.Parameter(torch.ones(1))
        self.spatial_mlp = nn.Linear(3, num_of_heads)
        nn.init.xavier_uniform_(self.spatial_mlp.weight)
    
    def forward(self, data):
        """Override forward to add spatial encoding"""
        in_nodes_features, edge_index, pos = data  # ADD pos
        num_of_nodes = in_nodes_features.shape[self.nodes_dim]
        
        # Dropout on input (from parent)
        in_nodes_features = self.dropout(in_nodes_features)
        
        # Feature projection (from parent)
        nodes_features_proj = self.linear_proj(in_nodes_features).view(
            -1, self.num_of_heads, self.num_out_features)
        nodes_features_proj = self.dropout(nodes_features_proj)
        
        # Attention scores (from parent)
        scores_source = (nodes_features_proj * self.scoring_fn_source).sum(dim=-1)
        scores_target = (nodes_features_proj * self.scoring_fn_target).sum(dim=-1)
        
        # Lift to edges (parent's method)
        scores_source_lifted, scores_target_lifted, nodes_features_proj_lifted = \
            self.lift(scores_source, scores_target, nodes_features_proj, edge_index)
        
        # ORIGINAL GAT scores
        scores_per_edge = scores_source_lifted + scores_target_lifted
        
        # YOUR ADDITION: Spatial encoding
        row, col = edge_index
        delta_pos = pos[col] - pos[row]
        spatial_scores = self.spatial_mlp(delta_pos)
        scores_per_edge = scores_per_edge + self.spatial_weight * spatial_scores
        
        # Apply activation
        scores_per_edge = self.leakyReLU(scores_per_edge)
        
        # Compute attention (parent's method)
        attentions_per_edge = self.neighborhood_aware_softmax(
            scores_per_edge, edge_index[self.trg_nodes_dim], num_of_nodes)
        attentions_per_edge = self.dropout(attentions_per_edge)
        
        # Aggregate (parent's method)
        nodes_features_proj_lifted_weighted = nodes_features_proj_lifted * attentions_per_edge
        out_nodes_features = self.aggregate_neighbors(
            nodes_features_proj_lifted_weighted, edge_index, 
            in_nodes_features, num_of_nodes)
        
        # Skip connections, concat, bias (parent's method)
        out_nodes_features = self.skip_concat_bias(
            attentions_per_edge, in_nodes_features, out_nodes_features)
        
        return (out_nodes_features, edge_index)

ModuleNotFoundError: No module named 'GAT'

# Test with dummy data

In [ ]:
# Create layer
layer = GATWithSpatial(
    num_in_features=3,  # xyz
    num_out_features=64,
    num_of_heads=4
)

# Dummy data
N = 100  # 100 points
features = torch.randn(N, 3)
pos = torch.randn(N, 3)
edge_index = torch.randint(0, N, (2, 500))  # 500 edges

# Forward pass
data = (features, edge_index, pos)
output, _ = layer(data)

print(f"Input shape: {features.shape}")
print(f"Output shape: {output.shape}")  # Should be [100, 256] (4 heads * 64 features)

NameError: name 'GATWithSpatial' is not defined